### Initialize notebook

In [1]:
# Torch imports
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.dataset import Dataset
import PIL.Image as IMG
from torch.utils.data.sampler import WeightedRandomSampler

# Other imports
import matplotlib.pyplot as plt
import numpy as np
import os
os.chdir('/home/ak/Spring2018/ature')
from neuralnet.utils.datasets import  DriveDatasetFromFile, DriveDatasetFromImageObj

import utils.img_utils as imgutil
from commons.IMAGE import SegmentedImage
from neuralnet.trainer import NNTrainer

sep = os.sep

# Define folders (create them if needed)
Dirs = {}
Dirs['train_data']      = 'data'+sep+'DRIVE'+sep+'training'+sep +'patches'

Dirs['data']      = 'data'+sep+'DRIVE'+sep+'test'
Dirs['images']    = Dirs['data'] +sep+ 'images'
Dirs['mask']      = Dirs['data'] +sep+ 'mask'
Dirs['truth']     = Dirs['data'] +sep+ '1st_manual'
Dirs['segmented'] = Dirs['data'] +sep+ 'drive_segmented'
Dirs['test_data'] = Dirs['data'] +sep+ 'patches'
Dirs['checkpoint']   = 'data' +sep+ 'checkpoint'

for k, folder in Dirs.items():
    os.makedirs(folder, exist_ok=True)

# Set up execution flags
Flags = {}
Flags['useGPU'] = False

classes = ('background', 'vessel')

### Define the network

In [2]:
class Net(nn.Module):
    def __init__(self, width, channels):
        super(Net, self).__init__()
        
        self.channels = channels
        self.width = width
        
    
        self.kern_size = 5
        self.kern_stride = 2      
        self.kern_padding = 2
        self.mxp_kern_size = 2
        self.mxp_stride = 2 
        self.pool1 = nn.MaxPool2d(kernel_size=self.mxp_kern_size, stride=self.mxp_stride)
        self.conv1 = nn.Conv2d(self.channels, 20, self.kern_size, 
                               stride=self.kern_stride, padding=self.kern_padding)
        self._update_output_size()
        
        
        self.kern_size = 5
        self.kern_stride = 1      
        self.kern_padding = 2
        self.mxp_kern_size = 1
        self.mxp_stride = 1 
        self.pool2 = nn.MaxPool2d(kernel_size=self.mxp_kern_size, stride=self.mxp_stride)
        self.conv2 = nn.Conv2d(20, 50, self.kern_size, 
                               stride=self.kern_stride, padding=self.kern_padding)
        self._update_output_size()
        
        
        self.kern_size = 5
        self.kern_stride = 1      
        self.kern_padding = 1
        self.mxp_kern_size = 1
        self.mxp_stride = 1 
        self.pool3 = nn.MaxPool2d(kernel_size=self.mxp_kern_size, stride=self.mxp_stride)
        self.conv3 = nn.Conv2d(50, 50, self.kern_size, 
                               stride=self.kern_stride, padding=self.kern_padding)
        self._update_output_size()
        
        
        
        self.linearWidth = 50*int(self.width)*int(self.width)
        self.fc1 = nn.Linear(self.linearWidth, 100)
        self.fc2 = nn.Linear(100, 20)
        self.fc3 = nn.Linear(20, 4)
        self.sm = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))
        x = x.view(-1, self.linearWidth)
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def _update_output_size(self):       
        temp = self.width
        self.width = ((self.width - self.kern_size + 2 * self.kern_padding) / self.kern_stride) + 1
        temp1 = self.width
        self.width = ((self.width - self.mxp_kern_size)/self.mxp_stride) + 1
        print('output width { ' + str(temp) + ' -conv-> ' + str(temp1) + ' -maxpool-> ' + str(self.width) + ' }')

width = 31
channels = 1
net = Net(width, channels)
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

output width { 31 -conv-> 16.0 -maxpool-> 8.0 }
output width { 8.0 -conv-> 8.0 -maxpool-> 8.0 }
output width { 8.0 -conv-> 6.0 -maxpool-> 6.0 }


### Load training dataset

In [3]:
transform = transforms.Compose([
    transforms.ToTensor()])

trainset = DriveDatasetFromFile(data_path=Dirs['train_data'], height=31, width=31,
                                num_classes=2, transform=transform)
clss, class_counts = np.unique(trainset.labels, return_counts=True)
class_weights = 1.0/class_counts
data_weights = np.array([class_weights[t] for t in trainset.labels]) 
second_min_class_count =  np.partition(class_counts, 1)[1]

sampler = WeightedRandomSampler(data_weights, int(10000))
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=False, num_workers=3, sampler=sampler)
print(second_min_class_count)

Data file loaded: data/DRIVE/training/patches/22_training.npy
183119


### Load test dataset

In [4]:
testset = DriveDatasetFromFile(data_path=Dirs['test_data'], height=31, width=31, 
                               transform=transform, num_classes=2)

clss_test, class_counts_test = np.unique(testset.labels, return_counts=True)
class_weights_test = 1.0/class_counts_test

data_weights_test = np.array([class_weights_test[t] for t in testset.labels])
data_weights_test = np.ones_like(testset.labels)

second_min_class_count_test =  np.partition(class_counts_test, 1)[1]

sampler_test = WeightedRandomSampler(data_weights_test, int(5000))
testloader = torch.utils.data.DataLoader(testset, batch_size=2, shuffle=False, num_workers=3, sampler=sampler_test)

Data file loaded: data/DRIVE/test/patches/02_test.npy


### Train and evaluate the network

In [5]:
trainer = NNTrainer(model=net, trainloader=trainloader, testloader=testloader, 
                    checkpoint_dir=Dirs['checkpoint'], checkpoint_file='checkpoint2Way.nn.tar')
trainer.train(optimizer=optimizer, epochs=1, use_gpu=Flags['useGPU'], override_checkpoint=False)
# trainer.resume_latest_model()
# trainer.test()

Starting training...
[epoch: 1, batches:   500] loss: 0.774
[epoch: 1, batches:  1000] loss: 0.710
[epoch: 1, batches:  1500] loss: 0.708
[epoch: 1, batches:  2000] loss: 0.704
[epoch: 1, batches:  2500] loss: 0.701
Done with training.
Accuracy of 500 batches: 96 %
Accuracy of 1000 batches: 96 %
Accuracy of 1500 batches: 96 %
Accuracy of 2000 batches: 96 %
Accuracy of 2500 batches: 96 %
Last checkpoint: checkpoint2Way.nn.tar
Last checkpoint was better.


### Check per-class performance

In [ ]:
class_correct = list(0. for i in range(2))
class_total = list(0. for i in range(2))
for data in testloader:
    images, labels = data
    outputs = net(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    c = (predicted == labels).squeeze()
    for i in range(2):
        label = labels[i]
        class_correct[label] += c[i]
        class_total[label] += 1

for i in range(2):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

### Convolve throughout the image to generate segmented image based on trained Network

In [ ]:
input_image = '19_test.tif'

def get_mask_file(file_name): 
    return file_name.split('_')[0] + '_test_mask.gif'

def get_ground_truth_file(file_name): 
    return file_name.split('_')[0] + '_manual1.gif'

img_obj = SegmentedImage()

img_obj.load_file(data_dir=Dirs['images'], file_name=input_image)
img_obj.res['orig'] = img_obj.image_arr[:, :, 1]
img_obj.working_arr = img_obj.image_arr[:, :, 1]
img_obj.working_arr = imgutil.whiten_image2d(img_obj.working_arr)

img_obj.load_mask(mask_dir=Dirs['mask'], fget_mask=get_mask_file, erode=True)
img_obj.load_ground_truth(gt_dir=Dirs['truth'], fget_ground_truth=get_ground_truth_file)

transform = transforms.Compose([transforms.ToTensor()])
dtest = DriveDatasetFromImageObj(img_obj=img_obj, patch_size=31, transform=transform, num_classes=2)
dclss, d_cls_count = np.unique(dtest.labels, return_counts=True)
dclass_weights = 1.0/d_cls_count
ddata_weights = np.array([dclass_weights[t] for t in dtest.labels]) 

dsampler = WeightedRandomSampler(ddata_weights, dtest.data.shape[0])
dtestloader = torch.utils.data.DataLoader(dtest, batch_size=1, shuffle=False, num_workers=2, sampler=dsampler)

In [ ]:
net.eval()
img_obj.res['seg'] = np.zeros_like(img_obj.working_arr)
for i, j, image, label in dtestloader:
    if Flags['useGPU']:
        images = image.cuda()
        label = label.cuda()
    outputs = net(Variable(image))
    _, predicted = torch.max(outputs.data, 1)
    img_obj.res['seg'][i, j] = predicted[0] * 255

In [ ]:
IMG.fromarray(img_obj.res['seg'])

In [ ]:
IMG.fromarray(img_obj.res['seg'])